In [10]:
import numpy 
import numba
from numba import jit
from numpy import *
from pylab import *

print(numba.__version__)

0.38.0


In [30]:
class MolecularDynamics:


    """Class that describes the molecular dynamics of a gas of atoms in units where m = epsilon = sigma = kb = 1"""
    
    dt = 0.001 # time increment
    sampleInterval = 100
    
    def __init__(self, N=4, L=10.0, initialTemperature=0.0):
    
        numpy.random.seed(219) # random number generator used for initial velocities (and sometimes positions) 
        
        self.N = N  # number of particles 
        self.L = L	# length of square side 
        self.initialTemperature = initialTemperature
        
        self.t = 0.0 # initial time
        self.tArray = array([self.t]) # array of time steps that is added to during integration
        self.steps = 0

        self.EnergyArray = array([]) # list of energy, sampled every sampleInterval time steps
        self.sampleTimeArray = array([])

        # accumulate statistics during time evolution
        self.temperatureArray = array([self.initialTemperature])
        self.temperatureAccumulator = 0.0
        self.squareTemperatureAccumulator = 0.0
        self.virialAccumulator = 0.0

        self.x = zeros(2*N) # NumPy array of N (x, y) positions
        self.v = zeros(2*N) # array of N (vx, vy) velocities

        self.xArray = array([]) # particle positions that is added to during integration
        self.vArray = array([]) # particle velocities
        
        self.forceType = "lennardJones"

In [5]:
    def minimumImage(self, x): # minimum image approximation (Gould Listing 8.2)

        L = self.L
        halfL = 0.5 * L

        return (x + halfL) % L - halfL

In [6]:
    def force(self): 

        if (self.forceType == "lennardJones"):
            f, virial = self.lennardJonesForce()

        if (self.forceType == "powerLaw"):
            f, virial = self.powerLawForce()

        self.virialAccumulator += virial

        return f

In [22]:
    def lennardJonesForce(self): # Gould Eq. 8.3 (NumPy vector form which is faster)

        N = self.N
        virial = 0.0
        tiny = 1.0e-40 # prevents division by zero in calculation of self-force
        L = self.L
        halfL = 0.5 * L

        x = self.x[arange(0, 2*N, 2)]
        y = self.x[arange(1, 2*N, 2)]
        f = zeros(2*N)

        minimumImage = self.minimumImage

        for i in range(N):  # The O(N**2) calculation that slows everything down SOS: ACCELERATE THIS LOOP

            dx = minimumImage(x[i] - x)
            dy = minimumImage(y[i] - y)
    
            r2inv = 1.0/(dx**2 + dy**2 + tiny)
            c = 48.0 * r2inv**7 - 24.0 * r2inv**4 # Where are epsilon and sigma parameters? We work in units where they r equal to 1?
            fx = dx * c
            fy = dy * c

            fx[i] = 0.0 # no self force
            fy[i] = 0.0
            f[2*i] = fx.sum()
            f[2*i+1] = fy.sum()

            virial += dot(fx, dx) + dot(fy, dy)

        return f, 0.5*virial

In [36]:
md = MolecularDynamics(N=16, L=4, initialTemperature=1.0)
%timeit md.lennardJonesForce(self)
    

AttributeError: 'MolecularDynamics' object has no attribute 'lennardJonesForce'

In [37]:
%timeit MolecularDynamics(N=16, L=4, initialTemperature=1.0)

18.5 µs ± 437 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:

%timeit MolecularDynamics(N=160, L=4, initialTemperature=1.0)

18.9 µs ± 310 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
%timeit MolecularDynamics(N=160, L=4, initialTemperature=1.0)

18.7 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [24]:
    def powerLawForce(self): 

        N = self.N
        virial = 0.0
        tiny = 1.0e-40 # prevents division by zero in calculation of self-force
        halfL = 0.5 * self.L

        x = self.x[arange(0, 2*N, 2)]
        y = self.x[arange(1, 2*N, 2)]	
        f = zeros(2*N)
        minimumImage = self.minimumImage
        for i in range(N):  # The O(N**2) calculation that slows everything down SOS: ACCELERATE THIS LOOP

            dx = minimumImage(x[i] - x)
            dy = minimumImage(y[i] - y)

            r2 = dx**2 + dy**2 + tiny
            r6inv = pow(r2, -3)
            fx = dx * r6inv
            fy = dy * r6inv

            fx[i] = 0.0 # no self force
            fy[i] = 0.0
            f[2*i] = fx.sum()
            f[2*i+1] = fy.sum()

            virial += dot(fx, dx) + dot(fy, dy)	

        return f, 0.5 * virial 